In [79]:
import pandas as pd
df = pd.read_csv('IIoT_Malware_Timeseries_CLEAN.csv') #creating pandas dataframe from the csv
df.head()

,Timestamp,Packet Size,Packet Length,Inter-Arrival Time,Protocol Type,Flags,Flow Duration,Total Packets,Total Bytes,Average Packet Size,...,Baseline Deviation,Packet Size Variance,Known IoC,C&C Communication,Data Exfiltration,Label,Threat Intensity,Base Risk,Modifier,Final Risk Score
0,2019-11-01 00:00:00,1500.0,1400.0,0.01,TCP,ACK,0.920496,150.0,80000.0,533.333333,...,0.038337,600.000000,0,0,0,Ransomware,High,1.00,0.00,1.00
1,2019-11-01 01:00:00,1500.0,1400.0,0.01,TCP,SYN,6.637806,150.0,80000.0,533.333333,...,0.166918,600.000000,0,0,1,Benign,Medium,0.66,-0.33,0.33
2,2019-11-01 02:00:00,1500.0,1400.0,0.01,TCP,SYN,5.666317,150.0,80000.0,533.333333,...,0.083234,251.082387,0,0,0,Benign,Medium,0.66,-0.33,0.33
3,2019-11-01 03:00:00,1500.0,1400.0,0.01,TCP,FIN,4.166854,150.0,80000.0,533.333333,...,0.221439,600.000000,0,0,0,Benign,Medium,0.66,-0.33,0.33
4,2019-11-01 04:00:00,1500.0,1400.0,0.01,TCP,FIN,2.179507,150.0,80000.0,533.333333,...,0.034485,600.000000,0,0,0,Benign,Low,0.33,-0.33,0.00


In [80]:
categorical_cols = ['Protocol Type', 'Flags'] #category columns
numerical_cols = ['Packet Size', 'Packet Length', 'Inter-Arrival Time','Flow Duration', 'Total Packets', 'Total Bytes',
    'Average Packet Size', 'Packet Arrival Rate',
    'Payload Entropy', 'Flow Entropy', 'Baseline Deviation', 'Packet Size Variance',
    'Known IoC', 'C&C Communication', 'Data Exfiltration'] #numerical columns
target_col= 'Label' #target column

In [81]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_cats = ohe.fit_transform(df[categorical_cols])
encoded_cat_columns = ohe.get_feature_names_out(categorical_cols)
df_encoded_cats = pd.DataFrame(encoded_cats, columns=encoded_cat_columns)
df_encoded_cats.index = df.index
df_numerical = df[numerical_cols]
x = pd.concat([df_numerical, df_encoded_cats], axis=1)
y = df[target_col]
x.shape, y.shape

((45289, 25), (45289,))

In [82]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2, random_state=42, stratify=y)
print(f'Train size is : {x_train.shape}')

Train size is : (36231, 25)


In [83]:
from sklearn.ensemble import RandomForestClassifier

model= RandomForestClassifier(n_estimators=700, max_depth= 23, random_state= 42, n_jobs=-1, class_weight='balanced' )
model.fit(x_train, y_train)

ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(x_test)
print("Accuracy is", accuracy_score(y_test, y_pred))
print("classification report is", classification_report(y_test, y_pred))

Accuracy is 0.6917641863546037
classification report is               precision    recall  f1-score   support

      Benign       0.70      0.99      0.82      6324
      Botnet       0.06      0.00      0.00       546
  Ransomware       0.03      0.00      0.00       739
     Spyware       0.20      0.01      0.01       649
      Trojan       0.14      0.01      0.01       449
        Worm       0.10      0.00      0.01       351

    accuracy                           0.69      9058
   macro avg       0.20      0.17      0.14      9058
weighted avg       0.52      0.69      0.57      9058



In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_train = x_train.fillna(0)
x_train_smote, y_train_smote = smote.fit_resample(x_train,y_train)
print("Original X_train shape:", x_train.shape)
print("Resampled X_train shape:", x_train_smote.shape)

Original X_train shape: (36231, 25)
Resampled X_train shape: (151782, 25)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model= RandomForestClassifier(n_estimators=1500, max_depth= 23, random_state= 42, n_jobs=-1, class_weight='balanced' )
model.fit(x_train_smote, y_train_smote)

RandomForestClassifier(class_weight='balanced', max_depth=23, n_estimators=700,
                       n_jobs=-1, random_state=42)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
x_test = x_test.fillna(0)
y_pred = model.predict(x_test)

print("Accuracy is", accuracy_score(y_test, y_pred))
print("Classification report is\n", classification_report(y_test, y_pred))

Accuracy is 0.554979024067123
Classification report is
               precision    recall  f1-score   support

      Benign       0.70      0.77      0.73      6324
      Botnet       0.07      0.03      0.05       546
  Ransomware       0.08      0.06      0.07       739
     Spyware       0.08      0.05      0.07       649
      Trojan       0.05      0.07      0.06       449
        Worm       0.03      0.02      0.02       351

    accuracy                           0.55      9058
   macro avg       0.17      0.17      0.17      9058
weighted avg       0.51      0.55      0.53      9058



In [ ]:
model_xgb = xgb.XGBClassifier(
    tree_method='gpu_hist',
    predictor='gpu_predictor',
    n_estimators=1000,        
    max_depth=7,              
    learning_rate=0.05,        
    gamma=5.0,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    reg_alpha = 1.0,
    reg_lambda=1.0,
    use_label_encoder=False,
    objective='multi:softprob',
    eval_metric='mlogloss'
)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


le.fit(pd.concat([y_train, y_test]))


y_train_smote_enc = le.transform(y_train_smote)
y_test_enc = le.transform(y_test)

In [ ]:
model_xgb.fit(x_train_smote, y_train_smote_enc)

C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:10:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:10:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=5.0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=-1,
              num_parallel_tree=None, ...)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred_xgb = model_xgb.predict(x_test)


y_pred_labels = le.inverse_transform(y_pred_xgb)


print("XGBoost Accuracy is", accuracy_score(y_test_enc, y_pred_xgb))
print("XGBoost Classification report is\n", classification_report(y_test_enc, y_pred_xgb))

C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:11:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


XGBoost Accuracy is 0.5973724884080371
XGBoost Classification report is
               precision    recall  f1-score   support

           0       0.70      0.84      0.76      6324
           1       0.09      0.03      0.04       546
           2       0.10      0.03      0.05       739
           3       0.04      0.01      0.02       649
           4       0.06      0.08      0.07       449
           5       0.04      0.03      0.04       351

    accuracy                           0.60      9058
   macro avg       0.17      0.17      0.16      9058
weighted avg       0.51      0.60      0.55      9058



In [ ]:
import numpy as np
from collections import Counter
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_curve, recall_score


print("Train distribution:", Counter(y_train))
print("Valid distribution:", Counter(y_valid))


sm = SMOTE(random_state=42)


classes = np.unique(y_train)

best_recall = {}
best_thresh = {}


for label in classes:
    print(f"\n=== Class '{label}' detector ===")
    
    
    y_bin     = (y_train == label).astype(int)
    y_val_bin = (y_valid == label).astype(int)
    
    
    if y_bin.sum() == 0:
        print(f"❌  No training samples for '{label}' → skipped")
        best_recall[label] = 0.0
        best_thresh[label] = None
        continue

    
    x_res, y_res = sm.fit_resample(x_train, y_bin)

  
    neg, pos = np.bincount(y_res)
    pos_weight = neg / pos

    
    clf = XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        use_label_encoder=False,
        eval_metric='logloss',
        scale_pos_weight=pos_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
    clf.fit(x_res, y_res)


    probs = clf.predict_proba(x_valid)[:, 1]
    p, r, t = precision_recall_curve(y_val_bin, probs)
    idxs = np.where(r >= 0.5)[0]
    if len(idxs):
        idx = idxs[0]
    else:
        idx = np.argmax(r)
    thresh = t[idx] if idx < len(t) else 0.5
    rec = r[idx]

    best_recall[label] = rec
    best_thresh[label] = thresh
    print(f"✅  Recall for '{label}': {rec:.2f} @ threshold {thresh:.2f}")


print("\n### Summary of per-family recall and thresholds ###")
for label in classes:
    print(f"{label:12s} → recall: {best_recall[label]:.2f}, threshold: {best_thresh[label]}")

Train distribution: Counter({'Benign': 20237, 'Ransomware': 2363, 'Spyware': 2076, 'Botnet': 1747, 'Trojan': 1438, 'Worm': 1123})
Valid distribution: Counter({'Benign': 5060, 'Ransomware': 591, 'Spyware': 519, 'Botnet': 437, 'Trojan': 359, 'Worm': 281})

=== Class 'Benign' detector ===


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:26

✅  Recall for 'Benign': 1.00 @ threshold 0.13

=== Class 'Botnet' detector ===


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Lo

✅  Recall for 'Botnet': 1.00 @ threshold 0.00

=== Class 'Ransomware' detector ===


...

C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


✅  Recall for 'Spyware': 1.00 @ threshold 0.00

=== Class 'Trojan' detector ===


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:26

✅  Recall for 'Trojan': 1.00 @ threshold 0.00

=== Class 'Worm' detector ===
✅  Recall for 'Worm': 1.00 @ threshold 0.00

### Summary of per-family recall and thresholds ###
Benign       → recall: 1.00, threshold: 0.12675392627716064
Botnet       → recall: 1.00, threshold: 0.00029300080495886505
Ransomware   → recall: 1.00, threshold: 0.00039489701157435775
Spyware      → recall: 1.00, threshold: 0.00023535384389106184
Trojan       → recall: 1.00, threshold: 5.238258381723426e-05
Worm         → recall: 1.00, threshold: 0.00010967328125843778


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


In [ ]:
import numpy as np
from collections import Counter
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_curve, recall_score


print("Train distribution:", Counter(y_train))
print("Valid distribution:", Counter(y_valid))


sm = SMOTE(random_state=42)


classes = np.unique(y_train)


classifiers = {}
for label in classes:
    print(f"\n--- Training detector for '{label}' ---")
    y_bin     = (y_train == label).astype(int)
    y_val_bin = (y_valid == label).astype(int)
    if y_bin.sum() == 0:
        print(f"No samples for '{label}', skipping.")
        continue

    
    x_res, y_res = sm.fit_resample(x_train, y_bin)
    neg, pos = np.bincount(y_res)
    pos_weight = neg / pos

    clf = XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        use_label_encoder=False,
        eval_metric='logloss',
        scale_pos_weight=pos_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
    clf.fit(x_res, y_res)
    classifiers[label] = clf
    print(f"'{label}' detector trained (pos_weight={pos_weight:.1f})")


min_recall = 0.50
min_precision = 0.10

chosen_thresholds = {}
results = {}

for label, clf in classifiers.items():
    print(f"\n--- Tuning threshold for '{label}' ---")
    y_val_bin = (y_valid == label).astype(int)
    probs     = clf.predict_proba(x_valid)[:,1]
    precisions, recalls, threshs = precision_recall_curve(y_val_bin, probs)

    
    valid_idxs = np.where((recalls >= min_recall) & (precisions >= min_precision))[0]
    if len(valid_idxs):
       
        best_idx = valid_idxs[np.argmax(precisions[valid_idxs])]
    else:
        
        f1_scores = 2 * precisions * recalls / (precisions + recalls + 1e-8)
        best_idx = np.argmax(f1_scores)

    thr = threshs[best_idx] if best_idx < len(threshs) else 0.5
    rec = recalls[best_idx]
    prec = precisions[best_idx]

    chosen_thresholds[label] = thr
    results[label] = {'recall': rec, 'precision': prec}
    print(f"Chosen thr={thr:.3f} → recall={rec:.2f}, precision={prec:.2f}")


print("\n=== Summary ===")
for label in classes:
    if label in results:
        r = results[label]
        print(f"{label:12s}: thr={chosen_thresholds[label]:.3f}, recall={r['recall']:.2f}, precision={r['precision']:.2f}")
    else:
        print(f"{label:12s}: no model trained")

Train distribution: Counter({'Benign': 20237, 'Ransomware': 2363, 'Spyware': 2076, 'Botnet': 1747, 'Trojan': 1438, 'Worm': 1123})
Valid distribution: Counter({'Benign': 5060, 'Ransomware': 591, 'Spyware': 519, 'Botnet': 437, 'Trojan': 359, 'Worm': 281})

--- Training detector for 'Benign' ---


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


'Benign' detector trained (pos_weight=1.0)

--- Training detector for 'Botnet' ---


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


'Botnet' detector trained (pos_weight=1.0)

--- Training detector for 'Ransomware' ---


...

C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [23:31:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


'Worm' detector trained (pos_weight=1.0)

--- Tuning threshold for 'Benign' ---
Chosen thr=0.582 → recall=0.54, precision=0.70

--- Tuning threshold for 'Botnet' ---
Chosen thr=0.016 → recall=0.96, precision=0.06

--- Tuning threshold for 'Ransomware' ---
Chosen thr=0.023 → recall=0.95, precision=0.08

--- Tuning threshold for 'Spyware' ---


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
C:\U

Chosen thr=0.028 → recall=0.93, precision=0.07

--- Tuning threshold for 'Trojan' ---
Chosen thr=0.325 → recall=0.25, precision=0.06

--- Tuning threshold for 'Worm' ---
Chosen thr=0.013 → recall=0.89, precision=0.04

=== Summary ===
Benign      : thr=0.582, recall=0.54, precision=0.70
Botnet      : thr=0.016, recall=0.96, precision=0.06
Ransomware  : thr=0.023, recall=0.95, precision=0.08
Spyware     : thr=0.028, recall=0.93, precision=0.07
Trojan      : thr=0.325, recall=0.25, precision=0.06
Worm        : thr=0.013, recall=0.89, precision=0.04


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:2676: UserWarning: [23:31:49] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  if len(data.shape) != 1 and self.num_features() != data.shape[1]:


In [ ]:
import numpy as np
from sklearn.metrics import recall_score, precision_score


probs_trojan = classifiers['Trojan'].predict_proba(x_valid)[:, 1]


y_val_trojan = (y_valid == 'Trojan').astype(int)


thresholds = np.concatenate([
    np.linspace(0.01, 0.10, 10),
    np.linspace(0.15, 0.50, 8)
])


results = []
for t in thresholds:
    preds = (probs_trojan >= t).astype(int)
    rec = recall_score(y_val_trojan, preds)
    prec = precision_score(y_val_trojan, preds, zero_division=0)
    results.append((t, rec, prec))
    print(f"thr={t:.3f} → recall={rec:.2f}, precision={prec:.2f}")


valid = [(t, rec, prec) for t, rec, prec in results if rec >= 0.50]
if valid:
    best_t, best_rec, best_prec = min(valid, key=lambda x: x[0])
    print(f"\nBest threshold for Trojan: {best_t:.3f} → recall={best_rec:.2f}, precision={best_prec:.2f}")
else:
    print("\nNo threshold in the tested range achieves 50% recall.")

thr=0.010 → recall=0.94, precision=0.05
thr=0.020 → recall=0.85, precision=0.05
thr=0.030 → recall=0.82, precision=0.05
thr=0.040 → recall=0.78, precision=0.05
thr=0.050 → recall=0.73, precision=0.05
thr=0.060 → recall=0.69, precision=0.05
thr=0.070 → recall=0.64, precision=0.05
thr=0.080 → recall=0.60, precision=0.05
thr=0.090 → recall=0.57, precision=0.05
thr=0.100 → recall=0.55, precision=0.05
thr=0.150 → recall=0.45, precision=0.05
thr=0.200 → recall=0.36, precision=0.05
thr=0.250 → recall=0.31, precision=0.06
thr=0.300 → recall=0.26, precision=0.06
thr=0.350 → recall=0.21, precision=0.06
thr=0.400 → recall=0.16, precision=0.06
thr=0.450 → recall=0.13, precision=0.06
thr=0.500 → recall=0.09, precision=0.06

Best threshold for Trojan: 0.010 → recall=0.94, precision=0.05


In [ ]:
from sklearn.metrics import recall_score, precision_score


chosen_thresholds['Trojan'] = 0.01
print("\n=== Final Summary with manual Trojan override ===")
for label, clf in classifiers.items():
    thr   = chosen_thresholds[label]
    y_bin = (y_valid == label).astype(int)
    probs = clf.predict_proba(x_valid)[:,1]
    preds = (probs >= thr).astype(int)
    
    rec  = recall_score(y_bin, preds)
    prec = precision_score(y_bin, preds, zero_division=0)
    print(f"{label:12s}: thr={thr:.3f}, recall={rec:.2f}, precision={prec:.2f}")


=== Final Summary with manual Trojan override ===
Benign      : thr=0.582, recall=0.54, precision=0.70
Botnet      : thr=0.016, recall=0.96, precision=0.06
Ransomware  : thr=0.023, recall=0.95, precision=0.08
Spyware     : thr=0.028, recall=0.93, precision=0.07
Trojan      : thr=0.010, recall=0.94, precision=0.05
Worm        : thr=0.013, recall=0.89, precision=0.04


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, recall_score, precision_score
min_recall    = 0.50
min_precision = 0.20   

new_thresholds = {}
new_results    = {}

print(f"▶️ Re-tuning thresholds for min_recall={min_recall}, min_precision={min_precision}\n")

for label, clf in classifiers.items():
    print(f"--- {label} ---")
    y_val_bin = (y_valid == label).astype(int)
    probs     = clf.predict_proba(x_valid)[:,1]
    precisions, recalls, threshs = precision_recall_curve(y_val_bin, probs)
    valid_idxs = np.where((recalls >= min_recall) & (precisions >= min_precision))[0]

    if len(valid_idxs):
        best_idx = valid_idxs[np.argmax(precisions[valid_idxs])]
    else:
        f1s      = 2 * precisions * recalls / (precisions + recalls + 1e-8)
        best_idx = np.argmax(f1s)

    thr  = threshs[best_idx] if best_idx < len(threshs) else 0.5
    rec  = recalls[best_idx]
    prec = precisions[best_idx]

    new_thresholds[label] = thr
    new_results[label]    = (rec, prec)
    print(f"thr={thr:.3f} → recall={rec:.2f}, precision={prec:.2f}")


print("\n=== New per-family summary ===")
for label in classifiers:
    rec, prec = new_results[label]
    print(f"{label:12s} → thr={new_thresholds[label]:.3f}, recall={rec:.2f}, precision={prec:.2f}")

▶️ Re-tuning thresholds for min_recall=0.5, min_precision=0.2

--- Benign ---
thr=0.582 → recall=0.54, precision=0.70
--- Botnet ---
thr=0.016 → recall=0.96, precision=0.06
--- Ransomware ---
thr=0.023 → recall=0.95, precision=0.08
--- Spyware ---
thr=0.028 → recall=0.93, precision=0.07
--- Trojan ---
thr=0.325 → recall=0.25, precision=0.06
--- Worm ---
thr=0.013 → recall=0.89, precision=0.04

=== New per-family summary ===
Benign       → thr=0.582, recall=0.54, precision=0.70
Botnet       → thr=0.016, recall=0.96, precision=0.06
Ransomware   → thr=0.023, recall=0.95, precision=0.08
Spyware      → thr=0.028, recall=0.93, precision=0.07
Trojan       → thr=0.325, recall=0.25, precision=0.06
Worm         → thr=0.013, recall=0.89, precision=0.04


In [ ]:
import numpy as np
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_curve, recall_score, precision_score


print(" Calibrating classifiers with sigmoid Platt scaling (3-fold CV)…")
calibrated_clfs = {}
for label, clf in classifiers.items():
    print(f"  • Calibrating '{label}'")
    calibrator = CalibratedClassifierCV(
        base_estimator=clf,
        method='isotonic',  
        cv=3
    )
    
    y_bin = (y_train == label).astype(int)
    calibrator.fit(x_train, y_bin)
    calibrated_clfs[label] = calibrator
print("  Calibration complete.\n")


classifiers = calibrated_clfs


min_recall    = 0.40
min_precision = 0.15

new_thresholds = {}
new_results    = {}

print(f" Re-tuning thresholds for min_recall={min_recall}, min_precision={min_precision}\n")
for label, clf in classifiers.items():
    print(f"--- {label} ---")
    y_val_bin = (y_valid == label).astype(int)
    probs     = clf.predict_proba(x_valid)[:,1]

    precisions, recalls, threshs = precision_recall_curve(y_val_bin, probs)
    valid_idxs = np.where((recalls >= min_recall) & (precisions >= min_precision))[0]

    if len(valid_idxs):
        best_idx = valid_idxs[np.argmax(precisions[valid_idxs])]
    else:
        f1s      = 2 * precisions * recalls / (precisions + recalls + 1e-8)
        best_idx = np.argmax(f1s)

    thr  = threshs[best_idx] if best_idx < len(threshs) else 0.5
    rec  = recalls[best_idx]
    prec = precisions[best_idx]

    new_thresholds[label] = thr
    new_results[label]    = (rec, prec)
    print(f"thr={thr:.3f} → recall={rec:.2f}, precision={prec:.2f}")

print("\n=== New per-family summary with calibration ===")
for label in classifiers:
    rec, prec = new_results[label]
    print(f"{label:12s} → thr={new_thresholds[label]:.3f}, recall={rec:.2f}, precision={prec:.2f}")

▶️ Calibrating classifiers with sigmoid Platt scaling (3-fold CV)…
  • Calibrating 'Benign'


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\calibration.py:300: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users

  • Calibrating 'Botnet'


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\calibration.py:300: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users

  • Calibrating 'Ransomware'


...

  • Calibrating 'Trojan'


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\calibration.py:300: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users

  • Calibrating 'Worm'


C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\calibration.py:300: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\anant\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [00:39:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users

✅  Calibration complete.

▶️ Re-tuning thresholds for min_recall=0.4, min_precision=0.15

--- Benign ---
thr=0.700 → recall=0.40, precision=0.85
--- Botnet ---
thr=0.063 → recall=0.41, precision=0.29
--- Ransomware ---
thr=0.086 → recall=0.43, precision=0.31
--- Spyware ---
thr=0.075 → recall=0.40, precision=0.28
--- Trojan ---
thr=0.052 → recall=0.45, precision=0.28
--- Worm ---
thr=0.042 → recall=0.41, precision=0.43

=== New per-family summary with calibration ===
Benign       → thr=0.700, recall=0.40, precision=0.85
Botnet       → thr=0.063, recall=0.41, precision=0.29
Ransomware   → thr=0.086, recall=0.43, precision=0.31
Spyware      → thr=0.075, recall=0.40, precision=0.28
Trojan       → thr=0.052, recall=0.45, precision=0.28
Worm         → thr=0.042, recall=0.41, precision=0.43


In [ ]:
from sklearn.metrics import classification_report
final_thresholds = {
    'Benign'     : 0.700,
    'Botnet'     : 0.063,
    'Ransomware' : 0.086,
    'Spyware'    : 0.075,
    'Trojan'     : 0.052,
    'Worm'       : 0.042
}


y_pred_final = []
for xi in x_valid_imp:
    #calibrated possibilities
    probs = {
        lbl: clf.predict_proba(xi.reshape(1, -1))[0,1]
        for lbl, clf in classifiers.items()
    }
    hits = [lbl for lbl, p in probs.items() if p >= final_thresholds[lbl]]
    if hits:
        y_pred_final.append(hits[0])
    else:
        #picking max probability family
        y_pred_final.append(max(probs, key=probs.get))

print(" Final per-family classification report:\n")
print(classification_report(y_valid, y_pred_final, digits=2))

🚀 Final per-family classification report:

              precision    recall  f1-score   support

      Benign       0.78      0.82      0.80      5060
      Botnet       0.32      0.36      0.34       437
  Ransomware       0.36      0.34      0.35       591
     Spyware       0.34      0.28      0.30       519
      Trojan       0.35      0.26      0.30       359
        Worm       0.43      0.28      0.34       281

    accuracy                           0.67      7247
   macro avg       0.43      0.39      0.40      7247
weighted avg       0.65      0.67      0.66      7247



In [ ]:
import joblib
joblib.dump(classifiers, 'family_detectors.pkl')
joblib.dump(final_thresholds, 'thresholds.json')

['thresholds.json']

: 